# Logistic Regression

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [3]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [4]:
W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = 1/(1+torch.exp(-(x_train.matmul(W))+b))

In [5]:
print(hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)


In [7]:
# pytorch에서는 sigmoid 함수 제공
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
print(hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)


In [8]:
# 오차 구하기
losses = -(y_train*torch.log(hypothesis)+(1-y_train)*torch.log(1-hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [9]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [11]:
# pytorch에서는 logistic regression의 cost 함수를 제공함
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

In [12]:
# 전체 코드
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([W,b],lr=1)

nb_epochs = 1000

for epoch in range(nb_epochs+1):
  hypothesis = torch.sigmoid(x_train.matmul(W)+b)
  # re = 1-hypothesis
  cost = -(y_train*torch.log(hypothesis)+(1-y_train)*torch.log(1-hypothesis)).mean()
  
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%100==0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [13]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
print(hypothesis)

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [14]:
prediction = hypothesis >=torch.FloatTensor([0.5])

print(prediction)

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


In [15]:
print(W,b)

tensor([[3.2530],
        [1.5179]], requires_grad=True) tensor([-14.4819], requires_grad=True)


# nn.Module로 구현

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [17]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# nn.Sequential()은 nn.Module 층을 차례로 쌓을 수 있도록 함
model = nn.Sequential(
    nn.Linear(2,1), # input_dim =2, output_dim = 1
    nn.Sigmoid() # 출력함수는 sigmoid
)

model(x_train)

tensor([[0.4020],
        [0.4147],
        [0.6556],
        [0.5948],
        [0.6788],
        [0.8061]], grad_fn=<SigmoidBackward0>)

## tensor에 저장된 값만 가져오려면 item() 써서 가져오기

In [19]:
optimizer = optim.SGD(model.parameters(),lr=1)

nb_epochs=1000
for epoch in range(nb_epochs+1):
  hypo = model(x_train)

  cost = F.binary_cross_entropy(hypo,y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%10==0:
    prediction = hypo >= torch.FloatTensor([0.5]) # 예측값이 0.5이상이면 True

    correct_pred = prediction.float() == y_train

    accu = correct_pred.sum().item()/len(correct_pred)

    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accu * 100,
        ))

Epoch    0/1000 Cost: 0.527002 Accuracy 83.33%
Epoch   10/1000 Cost: 0.994596 Accuracy 66.67%
Epoch   20/1000 Cost: 0.570399 Accuracy 83.33%
Epoch   30/1000 Cost: 0.473909 Accuracy 83.33%
Epoch   40/1000 Cost: 0.388606 Accuracy 83.33%
Epoch   50/1000 Cost: 0.307901 Accuracy 83.33%
Epoch   60/1000 Cost: 0.235923 Accuracy 83.33%
Epoch   70/1000 Cost: 0.182916 Accuracy 100.00%
Epoch   80/1000 Cost: 0.155795 Accuracy 100.00%
Epoch   90/1000 Cost: 0.143031 Accuracy 100.00%
Epoch  100/1000 Cost: 0.133369 Accuracy 100.00%
Epoch  110/1000 Cost: 0.124978 Accuracy 100.00%
Epoch  120/1000 Cost: 0.117600 Accuracy 100.00%
Epoch  130/1000 Cost: 0.111060 Accuracy 100.00%
Epoch  140/1000 Cost: 0.105223 Accuracy 100.00%
Epoch  150/1000 Cost: 0.099983 Accuracy 100.00%
Epoch  160/1000 Cost: 0.095251 Accuracy 100.00%
Epoch  170/1000 Cost: 0.090958 Accuracy 100.00%
Epoch  180/1000 Cost: 0.087044 Accuracy 100.00%
Epoch  190/1000 Cost: 0.083461 Accuracy 100.00%
Epoch  200/1000 Cost: 0.080169 Accuracy 100.00%

In [20]:
model(x_train)

tensor([[2.7553e-04],
        [3.1567e-02],
        [3.8923e-02],
        [9.5627e-01],
        [9.9824e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)

In [21]:
print(list(model.parameters()))

[Parameter containing:
tensor([[3.2543, 1.5186]], requires_grad=True), Parameter containing:
tensor([-14.4881], requires_grad=True)]


In [ ]:
# sigmoid 함수는 ANN의 hidden layer에서는 거의 사용되지 않는다.

# 클래스로 구현

In [22]:
class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(2,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    return self.sigmoid(self.linear(x))

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(),lr=1)

nb_epochs=1000

for epoch in range(nb_epochs+1):
  hypo = model(x_train)

  cost = F.binary_cross_entropy(hypo,y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%10==0:
    pred = hypo>=torch.FloatTensor([0.5])

    corr_pred = pred.float()==y_train

    accu = corr_pred.sum().item()/len(corr_pred)

    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accu * 100,
        ))

Epoch    0/1000 Cost: 0.614994 Accuracy 66.67%
Epoch   10/1000 Cost: 0.747550 Accuracy 83.33%
Epoch   20/1000 Cost: 0.633216 Accuracy 83.33%
Epoch   30/1000 Cost: 0.538123 Accuracy 83.33%
Epoch   40/1000 Cost: 0.450406 Accuracy 83.33%
Epoch   50/1000 Cost: 0.366382 Accuracy 83.33%
Epoch   60/1000 Cost: 0.287368 Accuracy 83.33%
Epoch   70/1000 Cost: 0.219289 Accuracy 83.33%
Epoch   80/1000 Cost: 0.173225 Accuracy 100.00%
Epoch   90/1000 Cost: 0.151674 Accuracy 100.00%
Epoch  100/1000 Cost: 0.140280 Accuracy 100.00%
Epoch  110/1000 Cost: 0.131002 Accuracy 100.00%
Epoch  120/1000 Cost: 0.122903 Accuracy 100.00%
Epoch  130/1000 Cost: 0.115765 Accuracy 100.00%
Epoch  140/1000 Cost: 0.109426 Accuracy 100.00%
Epoch  150/1000 Cost: 0.103760 Accuracy 100.00%
Epoch  160/1000 Cost: 0.098664 Accuracy 100.00%
Epoch  170/1000 Cost: 0.094056 Accuracy 100.00%
Epoch  180/1000 Cost: 0.089870 Accuracy 100.00%
Epoch  190/1000 Cost: 0.086050 Accuracy 100.00%
Epoch  200/1000 Cost: 0.082549 Accuracy 100.00%
